# Imports

In [7]:
from imutils.video import FPS
from imutils.video import FileVideoStream
import numpy as np
import argparse
import imutils
import time
import cv2
import os
from threading import Thread
import sys
import matplotlib.pyplot as plt
import multiprocessing
from multiprocessing import Queue, Pool
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

# import the Queue class from Python 3
if sys.version_info >= (3, 0):
    from queue import Queue
# otherwise, import the Queue class for Python 2.7
else:
    from Queue import Queue

# Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("models/research/")
sys.path.append("models/research/object_detection/")

PATH_TO_MODEL = "ssd_mobilenet_v1_coco_2017_11_17/frozen_inference_graph.pb"
PATH_TO_LABELS = "models/research/object_detection/data/mscoco_label_map.pbtxt"

NUM_CLASSES = 90

# Load label map

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES,
                                                           use_display_name = True)
category_index = label_map_util.create_category_index(categories)

# Detection sections ....

In [4]:
def detect_objects(image_np, sess, detection_graph):
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

    # Each box represents a part of the image where a particular object was detected.
    boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    scores = detection_graph.get_tensor_by_name('detection_scores:0')
    classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')

    # Actual detection.
    (boxes, scores, classes, num_detections) = sess.run(
        [boxes, scores, classes, num_detections],
        feed_dict={image_tensor: image_np_expanded})

    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=8)
    return image_np

In [5]:
def worker(input_q, output_q):
    # Load a (frozen) Tensorflow model into memory.
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')

        sess = tf.Session(graph=detection_graph)

    fps = FPS().start()
    while True:
        fps.update()
        frame = input_q.get()
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        output_q.put(detect_objects(frame_rgb, sess, detection_graph))

    fps.stop()
    sess.close()

In [ ]:
logger = multiprocessing.log_to_stderr()
logger.setLevel(multiprocessing.SUBDEBUG)

input_q = Queue(maxsize=5)
output_q = Queue(maxsize=5)
pool = Pool(2, worker, (input_q, output_q))

print("[INFO] starting video file thread...")
video_capture = FileVideoStream("../data/test_video.mp4").start()

time.sleep(1.0)
#video_capture = WebcamVideoStream(src=args.video_source,
#                                  width=args.width,
#                                  height=args.height).start()
fps = FPS().start()

while video_capture.more():  # fps._numFrames < 120
    frame = video_capture.read()
    frame = imutils.resize(frame, width=480)
    input_q.put(frame)

    t = time.time()

    output_rgb = cv2.cvtColor(output_q.get(), cv2.COLOR_RGB2BGR)
    #cv2.imshow('Video', output_rgb)
    clear_output(wait=True)
    plt.imshow(output_rgb)
    plt.show()
    fps.update()

    print('[INFO] elapsed time: {:.2f}'.format(time.time() - t))

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

fps.stop()
print('[INFO] elapsed time (total): {:.2f}'.format(fps.elapsed()))
print('[INFO] approx. FPS: {:.2f}'.format(fps.fps()))

pool.terminate()
video_capture.stop()
#cv2.destroyAllWindows()

[DEBUG/MainProcess] created semlock with handle 1995526144
[DEBUG/MainProcess] created semlock with handle 1995522048
[DEBUG/MainProcess] created semlock with handle 1995517952
[DEBUG/MainProcess] created semlock with handle 1995513856
[DEBUG/MainProcess] added worker
[DEBUG/MainProcess] added worker
[INFO/PoolWorker-1] child process calling self.run()
[INFO/PoolWorker-1] process shutting down
[INFO/PoolWorker-2] child process calling self.run()
[DEBUG/PoolWorker-1] running all "atexit" finalizers with priority >= 0
[INFO/PoolWorker-2] process shutting down
[DEBUG/PoolWorker-1] running the remaining "atexit" finalizers
Process PoolWorker-1:
[DEBUG/PoolWorker-2] running all "atexit" finalizers with priority >= 0
[DEBUG/PoolWorker-2] running the remaining "atexit" finalizers
Process PoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/p

[INFO] starting video file thread...


[DEBUG/MainProcess] cleaning up worker 0
[DEBUG/MainProcess] added worker
[DEBUG/MainProcess] added worker
[INFO/PoolWorker-3] child process calling self.run()
[INFO/PoolWorker-3] process shutting down
[INFO/PoolWorker-4] child process calling self.run()
[DEBUG/PoolWorker-3] running all "atexit" finalizers with priority >= 0
[INFO/PoolWorker-4] process shutting down
[DEBUG/PoolWorker-3] running the remaining "atexit" finalizers
[DEBUG/PoolWorker-4] running all "atexit" finalizers with priority >= 0
Process PoolWorker-3:
Traceback (most recent call last):
[DEBUG/PoolWorker-4] running the remaining "atexit" finalizers
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Process PoolWorker-4:
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/pool.py", line 97, in worker
    initializer(*initargs)
  File

[INFO/PoolWorker-13] child process calling self.run()
    self.run()
[INFO/PoolWorker-13] process shutting down
[DEBUG/PoolWorker-13] running all "atexit" finalizers with priority >= 0
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python2.7/multiprocessing/pool.py", line 97, in worker
[DEBUG/PoolWorker-13] running the remaining "atexit" finalizers
    initializer(*initargs)
Process PoolWorker-13:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "<ipython-input-5-de59c3e26f3b>", line 3, in worker
    self.run()
    detection_graph = tf.Graph()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
NameError: global name 'tf' is not defined
  File "/usr/lib/python2.7/multiprocessing/pool.py", line 97, in worker
[INFO/PoolWorker-12] process exiting with exitcode 1
 

[INFO/PoolWorker-22] child process calling self.run()
[INFO/PoolWorker-22] process shutting down
[DEBUG/PoolWorker-22] running all "atexit" finalizers with priority >= 0
[DEBUG/PoolWorker-22] running the remaining "atexit" finalizers
Process PoolWorker-22:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python2.7/multiprocessing/pool.py", line 97, in worker
    initializer(*initargs)
  File "<ipython-input-5-de59c3e26f3b>", line 3, in worker
    detection_graph = tf.Graph()
[DEBUG/MainProcess] cleaning up worker 0
NameError: global name 'tf' is not defined
[DEBUG/MainProcess] added worker
[INFO/PoolWorker-22] process exiting with exitcode 1
[INFO/PoolWorker-23] child process calling self.run()
[INFO/PoolWorker-23] process shutting down
[DEBUG/PoolWorker-23] running all "a

...

  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python2.7/multiprocessing/pool.py", line 97, in worker
    initializer(*initargs)
  File "<ipython-input-5-de59c3e26f3b>", line 3, in worker
    detection_graph = tf.Graph()
NameError: global name 'tf' is not defined
[INFO/PoolWorker-3145] process exiting with exitcode 1
[DEBUG/MainProcess] cleaning up worker 1
[DEBUG/MainProcess] added worker
[INFO/PoolWorker-3146] child process calling self.run()
[DEBUG/PoolWorker-3146] running all "atexit" finalizers with priority >= 0
[INFO/PoolWorker-3146] process shutting down
[DEBUG/PoolWorker-3146] running the remaining "atexit" finalizers
Process PoolWorker-3146:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7

    initializer(*initargs)
  File "<ipython-input-5-de59c3e26f3b>", line 3, in worker
    detection_graph = tf.Graph()
NameError: global name 'tf' is not defined
[INFO/PoolWorker-3154] process exiting with exitcode 1
[DEBUG/MainProcess] cleaning up worker 1
[DEBUG/MainProcess] added worker
[INFO/PoolWorker-3155] child process calling self.run()
[INFO/PoolWorker-3155] process shutting down
[DEBUG/PoolWorker-3155] running all "atexit" finalizers with priority >= 0
[DEBUG/PoolWorker-3155] running the remaining "atexit" finalizers
Process PoolWorker-3155:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python2.7/multiprocessing/pool.py", line 97, in worker
    initializer(*initargs)
  File "<ipython-input-5-de59c3e26f3b>", line 3, in worker
    detection_graph = tf.Graph()
Na

[INFO/PoolWorker-3164] child process calling self.run()
[INFO/PoolWorker-3164] process shutting down
[DEBUG/PoolWorker-3164] running all "atexit" finalizers with priority >= 0
[DEBUG/PoolWorker-3164] running the remaining "atexit" finalizers
Process PoolWorker-3164:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python2.7/multiprocessing/pool.py", line 97, in worker
    initializer(*initargs)
  File "<ipython-input-5-de59c3e26f3b>", line 3, in worker
    detection_graph = tf.Graph()
NameError: global name 'tf' is not defined
[INFO/PoolWorker-3164] process exiting with exitcode 1
[DEBUG/MainProcess] cleaning up worker 1
[DEBUG/MainProcess] added worker
[INFO/PoolWorker-3165] child process calling self.run()
[INFO/PoolWorker-3165] process shutting down
[DEBUG/PoolWorker-31

  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python2.7/multiprocessing/pool.py", line 97, in worker
    initializer(*initargs)
  File "<ipython-input-5-de59c3e26f3b>", line 3, in worker
    detection_graph = tf.Graph()
NameError: global name 'tf' is not defined
[INFO/PoolWorker-3173] process exiting with exitcode 1
[DEBUG/MainProcess] cleaning up worker 1
[DEBUG/MainProcess] added worker
[INFO/PoolWorker-3174] child process calling self.run()
[INFO/PoolWorker-3174] process shutting down
[DEBUG/PoolWorker-3174] running all "atexit" finalizers with priority >= 0
[DEBUG/PoolWorker-3174] running the remaining "atexit" finalizers
Process PoolWorker-3174:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7

    initializer(*initargs)
  File "<ipython-input-5-de59c3e26f3b>", line 3, in worker
    detection_graph = tf.Graph()
NameError: global name 'tf' is not defined
[INFO/PoolWorker-3182] process exiting with exitcode 1
[DEBUG/MainProcess] cleaning up worker 1
[DEBUG/MainProcess] added worker
[INFO/PoolWorker-3183] child process calling self.run()
[INFO/PoolWorker-3183] process shutting down
[DEBUG/PoolWorker-3183] running all "atexit" finalizers with priority >= 0
[DEBUG/PoolWorker-3183] running the remaining "atexit" finalizers
Process PoolWorker-3183:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python2.7/multiprocessing/pool.py", line 97, in worker
    initializer(*initargs)
  File "<ipython-input-5-de59c3e26f3b>", line 3, in worker
    detection_graph = tf.Graph()
Na

# Process video

In [ ]:
# start the file video stream thread and allow the buffer to
# start to fill
print("[INFO] starting video file thread...")
fvs = FileVideoStream("../data/test_video.mp4").start()
time.sleep(1.0)
 
# start the FPS timer
fps = FPS().start()
framecnt = 0
# loop over frames from the video file stream
while not fvs.stopped:
    framecnt = framecnt + 1
    # grab the frame from the threaded video file stream, resize
    # it, and convert it to grayscale (while still retaining 3
    # channels)
    frame = fvs.read()
    #frame = imutils.resize(frame, width=450)
    #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #frame = np.dstack([frame, frame, frame])

    # display the size of the queue on the frame
    #cv2.putText(frame, "Queue Size: {}".format(fvs.Q.qsize()),
    #    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)	

    # show the frame and update the FPS counter
    #cv2.imshow("Frame", frame)
    #cv2.waitKey(1)
    #clear_output(wait=True)
    #plt.imshow(frame)
    #plt.show()
    fps.update()
    #time.sleep(2)

# stop the timer and display FPS information
fps.stop()
print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
print("[INFO] Frames: {:.2f}".format(framecnt))
 
# do a bit of cleanup
#cv2.destroyAllWindows()
fps.stop()